In [8]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from tqdm import tqdm, trange

In [2]:
data = [
['Heute war ich mit meiner Familie Spazieren?', 'family'],
['Heute habe ich ein Buch gelesen, das hat mir sehr gefallen.', 'activity'],
['Heute war ein sehr stressiger Tag auf der Arbeit.', 'work'],
['Heute war mein Geburtstag! Ich habe ihn mit meiner Familie und Freunden gefeiert und es war ein unvergesslicher Tag.', 'family']]
df = pd.DataFrame(data, columns = ['text', 'category'])
df

,text,category
0,Heute war ich mit meiner Familie Spazieren?,family
1,"Heute habe ich ein Buch gelesen, das hat mir s...",activity
2,Heute war ein sehr stressiger Tag auf der Arbeit.,work
3,Heute war mein Geburtstag! Ich habe ihn mit me...,family


In [3]:
# Create Embeddings from data
text = df['text']
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
vectors = encoder.encode(text)

C:\Users\lmunz\PycharmProjects\MA_interactive\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# Build FAISS index from vectors
vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

In [7]:
def vector_search(query):
    search_vector = encoder.encode(query)
    _vector = np.array([search_vector])
    faiss.normalize_L2(_vector)
        
    k = index.ntotal
    distances, ann = index.search(_vector, k=k)
    
    results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
    
    return results

# search_text = 'Heute habe ich viel zeit mit meiner Familie verbracht.'
search_text = 'Heute habe ich Computer gespielt'
results = vector_search(search_text)

# Merge dfs for result presentation
merge = pd.merge(results, df, left_on='ann', right_index=True)
merge


,distances,ann,text,category
0,0.778020,1,"Heute habe ich ein Buch gelesen, das hat mir s...",activity
1,0.958226,3,Heute war mein Geburtstag! Ich habe ihn mit me...,family
2,0.966108,2,Heute war ein sehr stressiger Tag auf der Arbeit.,work
3,1.075433,0,Heute war ich mit meiner Familie Spazieren?,family
